In [ ]:
# -----------------------------------------------------------------------------
#
# OPF Relaxation Benchmark
#
# A Julia script to benchmark the performance, accuracy, and feasibility
# of various Optimal Power Flow (OPF) relaxations against the full AC-OPF.
#
# This script evaluates models on multiple scenarios and test cases from the
# PGLib-OPF library, generating a summary table and performance plots.
#
# -----------------------------------------------------------------------------

module OPFAnalysis

export run_analysis, print_summary_table, evaluate_scenarios

using PowerModels
using Ipopt
using JuMP
using Mosek
using MosekTools
using Random
using Statistics
using Printf
using JSON
using Distributions
using Clarabel
using SDPA
using Plots
using StatsPlots
using Logging
Logging.disable_logging(Logging.Warn)
# --- Configuration ---

"""
A dictionary mapping descriptive names to their PGLib file paths.
Ensure the `pglib-opf-master` directory is accessible from this script's location.
"""
const TEST_CASES = Dict(
    "case3" => "pglib-opf-master/pglib_opf_case3_lmbd.m",
    "case5" => "pglib-opf-master/pglib_opf_case5_pjm.m",
    "case14" => "pglib-opf-master/pglib_opf_case14_ieee.m",
    "case24" => "pglib-opf-master/pglib_opf_case24_ieee_rts.m",
    "case30" => "pglib-opf-master/pglib_opf_case30_ieee.m",
    "case118" => "pglib-opf-master/pglib_opf_case118_ieee.m",
    "case200" => "pglib-opf-master/pglib_opf_case200_activ.m",
    "case500"=> "pglib-opf-master/pglib_opf_case500_goc.m",
)

"""
A list of OPF formulations and their corresponding solvers to be benchmarked.
The format is ("AlgorithmName", PowerModels_Formulation, JuMP_Solver).
"""
const ALGORITHMS_TO_TEST = [
    ("ACOPF", ACPPowerModel, optimizer_with_attributes(Ipopt.Optimizer, "print_level" => 0)),
    ("DCOPF", DCPPowerModel, optimizer_with_attributes(Ipopt.Optimizer, "print_level" => 0)),
    ("SOC", SOCWRPowerModel, optimizer_with_attributes(Ipopt.Optimizer, "print_level" => 0)),
    ("QC", QCLSPowerModel, optimizer_with_attributes(Ipopt.Optimizer, "print_level" => 0)),
    ("SDP", SparseSDPWRMPowerModel, optimizer_with_attributes(Mosek.Optimizer, "MSK_IPAR_LOG" => 0)),
    ("LPACC", LPACCPowerModel, optimizer_with_attributes(Ipopt.Optimizer, "print_level" => 0))
]

# --- Core Functions ---

"""
    generate_scenarios(network_data::Dict, num_scenarios::Int, dist_params::Dict; seed::Int)

Generates multiple load scenarios by applying random scaling factors to the base loads.

# Arguments
- `network_data`: The base PowerModels network data dictionary.
- `num_scenarios`: The number of stochastic scenarios to generate.
- `dist_params`: A dictionary with parameters for the truncated normal distribution (`mean`, `sd`, `low`, `high`).
- `seed`: An integer for the random number generator to ensure reproducibility.

# Returns
- A `Vector` of network data dictionaries, one for each scenario.
"""

function generate_scenarios(network_data::Dict, num_scenarios::Int, dist_params::Dict; seed::Int)
    Random.seed!(seed)
    scenarios = []
    if !haskey(network_data, "bus") || isempty(network_data["bus"])
        error("Invalid network_data: 'bus' key missing or empty")
    end
    
    # --- FIX STARTS HERE ---
    # Create a dictionary to map actual bus IDs to scaling factors
    bus_ids = [bus["bus_i"] for (i, bus) in network_data["bus"]]
    # --- FIX ENDS HERE ---
    
    mean_val = get(dist_params, "mean", 1.0)
    sd = get(dist_params, "sd", 0.05)
    low = get(dist_params, "low", 0.9)
    high = get(dist_params, "high", 1.1)
    
    a, b = (low - mean_val) / sd, (high - mean_val) / sd
    dist = truncated(Normal(mean_val, sd), a, b)
    
    for _ in 1:num_scenarios
        # --- FIX STARTS HERE ---
        # Generate a random factor for each bus ID
        scaling_factors_map = Dict(zip(bus_ids, rand(dist, length(bus_ids))))
        # --- FIX ENDS HERE ---

        scen_data = deepcopy(network_data)
        for (l, load) in scen_data["load"]
            bus_id = load["load_bus"]
            # --- FIX STARTS HERE ---
            # Look up the scaling factor in the map
            if haskey(scaling_factors_map, bus_id)
                factor = scaling_factors_map[bus_id]
                scen_data["load"][l]["pd"] *= factor
                scen_data["load"][l]["qd"] *= factor
            end
            # --- FIX ENDS HERE ---
        end
        push!(scenarios, scen_data)
    end
    return scenarios
end

"""
    check_ac_feasibility(network_data::Dict, solution::Dict)

Checks the AC feasibility of a solution from a relaxation by running a power flow.

It fixes the generator active power setpoints from the solution and solves an AC power flow
to determine the resulting voltage and branch flow violations.

# Arguments
- `network_data`: The original network data for the scenario.
- `solution`: The solution dictionary from an OPF relaxation.

# Returns
- A tuple `(max_v_viol, max_line_viol)` containing the maximum per-unit voltage
  violation and the maximum percentage line rating violation. Returns `(NaN, NaN)` if
  the power flow fails to converge.
"""
function check_ac_feasibility(network_data::Dict, solution::Dict)
    if !haskey(solution, "gen") return (NaN, NaN) end

    pf_data = deepcopy(network_data)
    for (i, gen) in solution["gen"]
        if haskey(pf_data["gen"], i)
            pf_data["gen"][i]["pg"] = gen["pg"]
            pf_data["gen"][i]["vg"] = get(gen, "vg", 1.0)
        end
    end

    solver = optimizer_with_attributes(Ipopt.Optimizer, "print_level" => 0)
    pf_result = solve_pf(pf_data, ACPPowerModel, solver)

    max_v_viol = 0.0
    if haskey(pf_result, "solution") && haskey(pf_result["solution"], "bus")
        for (b, bus) in network_data["bus"]
            vm = pf_result["solution"]["bus"][b]["vm"]
            viol = max(0, bus["vmin"] - vm, vm - bus["vmax"])
            max_v_viol = max(max_v_viol, viol)
        end
    else
        return (NaN, NaN)
    end

    max_line_viol = 0.0
    if haskey(pf_result, "solution") && haskey(pf_result["solution"], "branch")
        s = pf_result["solution"]
        for (br, branch) in network_data["branch"]
             if haskey(branch, "rate_a")
                p_fr = s["branch"][br]["pf"]; q_fr = s["branch"][br]["qf"]
                p_to = s["branch"][br]["pt"]; q_to = s["branch"][br]["qt"]
                s_fr = sqrt(p_fr^2 + q_fr^2)
                s_to = sqrt(p_to^2 + q_to^2)
                viol = max(0, s_fr - branch["rate_a"], s_to - branch["rate_a"]) / branch["rate_a"] * 100
                max_line_viol = max(max_line_viol, viol)
            end
        end
    end
    
    return max_v_viol, max_line_viol
end

"""
    evaluate_scenarios(test_cases::Dict, num_scenarios::Int, algorithms::Vector; seed::Int)

Runs the core evaluation across all test cases and scenarios.

# Arguments
- `test_cases`: Dictionary mapping case names to file paths.
- `num_scenarios`: The number of scenarios to run per case.
- `algorithms`: A vector of algorithm tuples to test.
- `seed`: The random seed for scenario generation.

# Returns
- A `Dict` containing the summarized statistics for each test case and algorithm.
"""
function evaluate_scenarios(test_cases::Dict, num_scenarios::Int, algorithms::Vector; seed::Int)
    dist_params = Dict("mean" => 1.0, "sd" => 0.1, "low" => 0.7, "high" => 1.3)
    summary = Dict()

    for (case_name, case_file) in test_cases
        println("\nProcessing test case: $case_name")
        if !isfile(case_file)
            @warn "Test case file not found: $case_file. Skipping."
            continue
        end
        network_data = parse_file(case_file)
        scenarios = generate_scenarios(network_data, num_scenarios, dist_params; seed=seed)
        
        raw_results = Dict(name => Dict("gap"=>[], "runtime"=>[], "max_feas_viol"=>[]) for (name,_,_) in algorithms)
        success_counts = Dict(name => 0 for (name,_,_) in algorithms)

        for (i, scen_data) in enumerate(scenarios)
            ac_result = solve_opf(scen_data, ACPPowerModel, Ipopt.Optimizer)
            if ac_result["termination_status"] ∉ [OPTIMAL, LOCALLY_SOLVED]
                @warn "Scenario $i: ACOPF failed, skipping scenario."
                continue
            end
            success_counts["ACOPF"] += 1
            ac_obj = ac_result["objective"]
            push!(raw_results["ACOPF"]["runtime"], ac_result["solve_time"])

            for (algo_name, model, solver) in algorithms
                if algo_name == "ACOPF" continue end
                result = solve_opf(scen_data, model, solver)
                if result["termination_status"] ∈ [OPTIMAL, LOCALLY_SOLVED]
                    success_counts[algo_name] += 1
                    gap = (result["objective"] - ac_obj) / abs(ac_obj) * 100
                    max_v, max_l = check_ac_feasibility(scen_data, result["solution"])
                    max_feas = isnan(max_v) || isnan(max_l) ? NaN : max(max_v, max_l / 100.0)

                    push!(raw_results[algo_name]["gap"], gap)
                    push!(raw_results[algo_name]["runtime"], result["solve_time"])
                    push!(raw_results[algo_name]["max_feas_viol"], max_feas)
                end
            end
        end

        case_summary = Dict()
        for (algo_name, _, _) in algorithms
            filter_nan(x) = filter(!isnan, x)
            runtimes = filter_nan(raw_results[algo_name]["runtime"])
            gaps = filter_nan(raw_results[algo_name]["gap"])
            feas_viols = filter_nan(raw_results[algo_name]["max_feas_viol"])

            case_summary[algo_name] = Dict(
                "success_rate" => success_counts[algo_name] / num_scenarios * 100,
                "runtime_mean" => isempty(runtimes) ? NaN : mean(runtimes),
                "runtime_std"  => length(runtimes) > 1 ? std(runtimes) : 0.0,
                "gap_mean" => isempty(gaps) ? NaN : mean(gaps),
                "gap_std"  => length(gaps) > 1 ? std(gaps) : 0.0,
                "feas_viol_mean" => isempty(feas_viols) ? NaN : mean(feas_viols),
                "feas_viol_std"  => length(feas_viols) > 1 ? std(feas_viols) : 0.0,
            )
        end
        summary[case_name] = case_summary
    end
    return summary
end

# --- Reporting and Visualization ---

"""
    print_summary_table(summary::Dict)

Prints a formatted table of results to the console.
"""
function print_summary_table(summary::Dict)
    println("\n" * "="^122)
    println(rpad("Test Case", 15), rpad("Algorithm", 12), rpad("Success(%)", 12), rpad("Runtime(s) Avg", 18), rpad("Runtime(s) Std", 18), rpad("Gap(%) Avg", 15), rpad("Gap(%) Std", 15), "Avg Feas Viol(pu)")
    println("-"^122)

    for (case, case_summary) in sort(collect(summary), by=x->x[1])
        for (algo, _, _) in ALGORITHMS_TO_TEST
            if !haskey(case_summary, algo) continue end
            stats = case_summary[algo]
            
            success_str = @sprintf("%.1f", stats["success_rate"])
            rt_m_str = isnan(stats["runtime_mean"]) ? "N/A" : @sprintf("%.4f", stats["runtime_mean"])
            rt_s_str = isnan(stats["runtime_std"]) ? "N/A" : @sprintf("%.4f", stats["runtime_std"])
            g_m_str = isnan(stats["gap_mean"]) ? "N/A" : @sprintf("%.4f", stats["gap_mean"])
            g_s_str = isnan(stats["gap_std"]) ? "N/A" : @sprintf("%.4f", stats["gap_std"])
            
            feas_v_str = haskey(stats, "feas_viol_mean") && !isnan(stats["feas_viol_mean"]) ? 
                         @sprintf("%.2e", stats["feas_viol_mean"]) : "N/A"

            color = stats["success_rate"] >= 90.0 ? "\e[32m" : stats["success_rate"] >= 50.0 ? "\e[33m" : "\e[31m"
            
            println(rpad(case, 15), rpad(algo, 12), color, rpad(success_str, 12), "\e[0m", rpad(rt_m_str, 18), rpad(rt_s_str, 18), rpad(g_m_str, 15), rpad(g_s_str, 15), feas_v_str)
        end
        println("-"^122)
    end
end

"""
    create_enhanced_visualizations(summary::Dict, results_dir::String)

Generates and saves a 2x2 summary plot of the benchmark results.
"""
function create_visualizations(summary::Dict, results_dir::String)
    cases = sort(collect(keys(summary)))
    if isempty(cases)
        @warn "No valid results to plot. Skipping visualization."
        return
    end
    
    algorithms = [a[1] for a in ALGORITHMS_TO_TEST if a[1] != "ACOPF"]
    colors = palette(:Paired_6)
    
    # Plot 1: Feasibility Profile
    feas_data = [summary[c][a]["feas_viol_mean"] for c in cases, a in algorithms]
    floor_val = 1e-6 # Floor for log scale
    replace!(feas_data, NaN => floor_val)
    feas_data .= max.(feas_data, floor_val)
    p1 = groupedbar(cases, feas_data, labels=reshape(algorithms, 1, :), title="AC Feasibility Violation", ylabel="Max Violation (p.u.) [log scale]", yscale=:log10, color=colors[1:length(algorithms)]', legend=:outertopright, bar_width=0.7, xrotation=45, lw=0)
    hline!(p1, [0.05], linestyle=:dash, color=:red, label="5% Limit", lw=2)

    # Plot 2: Optimality Gap Profile
    gap_data = [summary[c][a]["gap_mean"] for c in cases, a in algorithms]
    replace!(gap_data, NaN => 0)
    p2 = groupedbar(cases, gap_data, labels=reshape(algorithms, 1, :), title="Optimality Gap vs. ACOPF", ylabel="Gap (%)", color=colors[1:length(algorithms)]', legend=false, bar_width=0.7, xrotation=45, lw=0)

    # Plot 3: Runtime Profile
    runtime_mean_data = [summary[c][a]["runtime_mean"] for c in cases, a in algorithms]
    runtime_std_data = [summary[c][a]["runtime_std"] for c in cases, a in algorithms]
    replace!(runtime_mean_data, NaN => 0)
    replace!(runtime_std_data, NaN => 0)
    p3 = groupedbar(cases, runtime_mean_data, yerror=runtime_std_data, labels=reshape(algorithms, 1, :), title="Solver Runtime (with ±1 Std Dev)", ylabel="Time (s) [log scale]", yscale=:log10, color=colors[1:length(algorithms)]', legend=false, bar_width=0.7, xrotation=45, lw=0)

    # Plot 4: Aggregate Trade-off Bubble Chart
    robust_mean(values) = (valid = filter(!isnan, values); isempty(valid) ? NaN : mean(valid))
    avg_runtime = [robust_mean([summary[c][a]["runtime_mean"] for c in cases]) for a in algorithms]
    avg_gap = [robust_mean([summary[c][a]["gap_mean"] for c in cases]) for a in algorithms]
    avg_feas = [robust_mean([summary[c][a]["feas_viol_mean"] for c in cases]) for a in algorithms]
    bubble_sizes = 5 .+ 300 .* replace(avg_feas, NaN => 0)
    
    p4 = plot(title="Trade-off: Speed vs. Quality vs. Feasibility", xlabel="Average Runtime (s) [log scale]", ylabel="Average Optimality Gap (%)", xscale=:log10, legend=:outertopright)
    scatter!(p4, [1e-4], [0], markercolor=:grey, markershape=:star5, markersize=8, label="Ideal")
    scatter!(p4, avg_runtime, avg_gap, markercolor=colors[1:length(algorithms)]', markersize=bubble_sizes, label=reshape(algorithms, 1, :), markershape=:circle, alpha=0.7, markerstrokewidth=2)

    # Combine and Save
    final_plot = plot(p1, p2, p3, p4, layout=(2, 2), size=(1600, 1200), margin=10Plots.mm, plot_title="OPF Relaxation Performance Summary")
    
    try
        if !isdir(results_dir) mkdir(results_dir) end
        path = joinpath(results_dir, "opf_benchmark_summary.png")
        savefig(final_plot, path)
        println("\n✅ Saved enhanced summary plots to $path")
    catch e
        @error "Error saving plot: $e"
    end
end

"""
    run_analysis(; num_scenarios::Int=10, seed::Int=2025)

Main function to execute the entire benchmarking analysis.
"""
function run_analysis(; num_scenarios::Int=10, seed::Int=2025)
    println("--- Starting OPF Relaxation Benchmark ---")
    
    # Check for PGLib directory
    if !isdir("pglib-opf-master")
        @error "PGLib directory 'pglib-opf-master' not found. Please download and place it in the same directory as this script."
        return
    end

    results_dir = "results"
    
    summary_results = evaluate_scenarios(TEST_CASES, num_scenarios, ALGORITHMS_TO_TEST; seed=seed)
    
    if isempty(summary_results)
        @error "Evaluation failed to produce any results."
        return
    end

    print_summary_table(summary_results)
    create_visualizations(summary_results, results_dir)

    println("\n--- Benchmark Complete ---")
end

end

 OPFAnalysis.run_analysis(num_scenarios=5, seed=2025)


--- Starting OPF Relaxation Benchmark ---

Processing test case: case3
[info | PowerModels]: removing 3 cost terms from generator 3: Float64[]
This is Ipopt version 3.14.17, running with linear solver MUMPS 5.8.0.

Number of nonzeros in equality constraint Jacobian...:       78
Number of nonzeros in inequality constraint Jacobian.:       18
Number of nonzeros in Lagrangian Hessian.............:      134

Total number of variables............................:       23
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       20
                     variables with only upper bounds:        0
Total number of equality constraints.................:       19
Total number of inequality constraints...............:        9
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        3
        inequality constraints with only upper bounds:        6

iter    objecti